In [1]:
import os
import numpy as np
import sys
import socket
import pickle
import math
import os
import json
from tqdm import tqdm
import torch
import k3d

from pytorch3d.io import load_obj,save_ply
from pytorch3d.ops import sample_points_from_meshes,knn_points
from pytorch3d.structures import Pointclouds,Meshes

In [2]:
pix_path = '/scratch/fml35/datasets/pix3d_new/'
with open("/data/cornucopia/fml35/experiments/exp_024_debug/models/model_list.json",'r') as f:
        model_list = json.load(f)["models"]

for j in tqdm(range(0,1)):#len(model_list))):
    path = pix_path + model_list[j]["model"]

100%|██████████| 1/1 [00:00<00:00, 3741.57it/s]


In [3]:
verts_torch,faces_torch,_ = load_obj(path,load_textures=False)

verts = verts_torch.numpy()
faces = faces_torch[0].numpy()

print(np.max(verts,axis=0))
print(np.min(verts,axis=0))
print('verts',verts.shape)

[0.28645244 0.14048965 0.38497746]
[-0.28645244 -0.14048965 -0.38497746]
verts (137, 3)


In [4]:
# line_segs = np.zeros((faces.shape[0]*3,6))
# counter = 0
# checked_pairs = []
# for i in range(faces.shape[0]):
#     for pair in [[0,1],[1,2],[2,0]]:
#         indices = [faces[i,pair[0]],faces[i,pair[1]]]
#         if indices not in checked_pairs and indices[::-1] not in checked_pairs:
#             line_segs[counter] = np.concatenate((verts[indices[0]],verts[indices[1]]))
#             checked_pairs.append(indices)
#             counter += 1
# line_segs = line_segs[:counter]

In [5]:
# dists = np.linalg.norm(line_segs[:,:3] - line_segs[:,3:6],axis=1)
# mask = dists > 0.9 * np.max(dists)
# long_lines = line_segs[np.where(mask == True)]
# print(long_lines)
# print(np.linalg.norm(long_lines[:,:3] - long_lines[:,3:6],axis=1))

In [6]:
# plot = k3d.plot()

# plt_mesh = k3d.mesh(verts, faces)
# plot += plt_mesh

# # for line in line_segs[:10]:
# #     print(line)
# # plot += k3d.line([line_segs[:,:3], line_segs[:,3:6]], shader='mesh', width=.003, color=0xff0000)
# plot += k3d.line([long_lines[0,:3], long_lines[0,3:6]], shader='mesh', width=.003, color=0xff0000)
# plot += k3d.line([long_lines[1,:3], long_lines[1,3:6]], shader='mesh', width=.003, color=0xff0000)
# plot += k3d.line([long_lines[2,:3], long_lines[2,3:6]], shader='mesh', width=.003, color=0xff0000)

# plot += k3d.points(verts,point_size=0.01,color=(0 + 256*255))
# plot.display()

In [7]:
n_points = 50000
n_nn = 5
angle_threshold_degree = 20


mesh = Meshes(verts=[verts_torch],faces=[faces_torch[0]])
sample_points,sample_normals = sample_points_from_meshes(mesh,n_points,return_normals=True)
sample_points = sample_points.squeeze()
sample_normals = sample_normals.squeeze()

# labels = np.empty(n_points,dtype=bool)

# for i in tqdm(range(sample_points.shape[0])):
#     dist = torch.norm(sample_points[i].tile(n_points,1) - sample_points, dim=1, p=None)
#     knn = dist.topk(n_nn, largest=False)

#     current_normal = sample_normals[i].tile(n_nn,1)
#     nn_normal = sample_normals[knn.indices]

#     # print('current_normal.shape',current_normal.shape)
#     # print('nn_normal',nn_normal.shape)
#     # print('current_normal.shape',current_normal)
#     # print('nn_normal',nn_normal)


#     normal_dot = torch.sum(current_normal*nn_normal,dim=1)
#     # print(normal_dot.shape)
#     # print('normal_dot',normal_dot)
#     angles = torch.arccos(normal_dot) * 180 / np.pi
#     angles = torch.min(angles,torch.abs(180. - angles))
#     # TODO: think if better 180 or 360
#     # angles = torch.min(angles,torch.abs(360. - angles))
#     # print(angles)
#     labels[i] = (torch.max(angles) > angle_threshold_degree).item()

In [8]:

# find distance between two pointclouds
# For each predicted point, find its neareast-neighbor GT point
knn_pred = knn_points(sample_points.unsqueeze(0),sample_points.unsqueeze(0), K=n_nn)

sample_normals_unsq = sample_normals.unsqueeze(1)
sample_normals_tiled = sample_normals_unsq.tile(1,n_nn,1)

nn_normals = torch.zeros(n_points,n_nn,3)
for i in tqdm(range(n_points)):
    n = sample_normals[knn_pred.idx[0,i]]
    nn_normals[i] = n
    
normal_dot = torch.sum(sample_normals_tiled*nn_normals,dim=2)
angles = torch.arccos(normal_dot) * 180 / np.pi
angles = torch.min(angles,torch.abs(180. - angles))
# TODO: think if better 180 or 360
# angles = torch.min(angles,torch.abs(360. - angles))
max_angles,_ = torch.max(angles,dim=1)
labels = (max_angles > angle_threshold_degree)
labels = labels.numpy()
print(labels.shape)



100%|██████████| 50000/50000 [00:00<00:00, 61540.54it/s]

(50000,)


In [9]:

plot = k3d.plot()

# plot += k3d.points(sample_points.numpy()[~labels],point_size=0.01,color=(255))
plot += k3d.points(sample_points.numpy()[labels],point_size=0.01,color=(0 + 256*255))
plot.display()

Output()

In [10]:
# maybe just do an optimisation, only free parameter is number of lines optimise by adjusting for every line what are the start coordinates and end coordinates
# minimise the distance of all points to the nearest line, constrain line endpoints to lie within reasonable volume